In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# ugly hack to make the imports work properly
# import os
# os.chdir('./paperpi')

In [3]:
from paperpi.library import Plugin, CacheFiles, get_help
from importlib import import_module
from pathlib import Path
import paperpi.my_constants as paperpi_constants
import logging
from IPython.display import Image 
import argparse
import sys


In [4]:
logger = logging.getLogger(__name__)

In [5]:
def setup_plugins(project_root, plugin_list=None, resolution=(640, 400)):
    '''create dictionary of plugins using sample configurations provided in each plugin directory
    
    Args:
        project_root(`str`): directory where main script is located
        use_all_layouts(`Bool`): use all discovered layouts
        plugin_list(`list`): list of plugins to update; when None all discovered plugins will be updated
        resolution(`tuple` of `int`): screen resolution to use when generating images
        
    Returns:
        `dict` of Plugin objects and layout name used to generate the Plugin
        '''
            
    plugin_path = Path(f'{project_root}/{paperpi_constants.plugins}')
    
    # discover all plugins
    plugins = get_help.get_modules(plugin_path)
    
    # reduce the list down to just the specified plugins
    if plugin_list:
        my_list = []
        for i in plugins:
            if i in plugin_list:
                my_list.append(i)
            else:
                print(f'specified plugin not found: {i}')
        plugins = my_list
    
    cache = CacheFiles()
    
    plugin_dict = {}
    
    for plugin in plugins:
        plugin_dict[plugin] = []
        all_layouts = {}
        print(f'found up plugin {plugin}')
        
        pkg_name = f'{".".join(plugin_path.parts)}'
        logging.debug(f'importing pkg: {pkg_name}')
        
        # import the plugin, layout and sample configuration
        try:
            module = import_module(f'{pkg_name}.{plugin}')            
            layout_import =  import_module(f'{pkg_name}.{plugin}.layout')
            sample_import = import_module(f'{pkg_name}.{plugin}.sample')
        except ModuleNotFoundError as e:
            print(f'skipping plugin {plugin} due to previous error: {e}')
            continue
        
        # get all of the layout dictionarys from the layout file
        for a in dir(layout_import):
            if not a.startswith('_') and isinstance(getattr(layout_import, a), dict):
                all_layouts[a] = (getattr(layout_import, a))
                
        if len(all_layouts) < 1:
            print(f'skipping plugin {plugin} due to missing layouts')
            continue
            
        # remove the default layout from the list if there are multiple layouts defined
#         if len(all_layouts) > 1 and 'layout' in all_layouts:
#             all_layouts.pop('layout')
        
        # make sure there is a valid configuration:
        try:
            config = sample_import.config
        except AttributeError as e:
            print(f'skipping plugin {plugin} due to missing sample configuration: {e}')
            continue
        
        # setup plugin
        for name, layout in all_layouts.items():
            print(f'adding layout: {name}')           
            my_plugin = Plugin(resolution=resolution,
                               cache=cache,
                               layout=layout,
                               update_function=module.update_function,
                               config=config
                              )
            my_plugin.refresh_rate = 1
        
            try:
                if 'kwargs' in config:
                    my_plugin.update(**config['kwargs'])

                else:
                    my_plugin.update()
            except Exception as e:
                print(f'plugin "{plugin}" could not be configured due to errors: {e}')
            plugin_dict[plugin].append({
                                   'plugin': plugin,
                                   'module': module, 
                                   'plugin_obj': my_plugin, 
                                   'doc_path': plugin_path/plugin,
                                   'layout': name})
    return plugin_dict


In [8]:
def create_readme(plugin_dict, project_root, overwrite_images=False):
    plugin_docs = {}
    plugin_path = Path(f'{project_root}/{paperpi_constants.plugins}')
    readme_name = 'README'
    readme_additional = '_additional'
    readme_suffix = '.md'
    
    # run through each plugin and gather the document information
    for plugin, layouts in plugin_dict.items():
        print(f'Processing plugins: {plugin}')
        try:
            doc_path = Path(layouts[0]['doc_path'])
            plugin_readme = Path(doc_path)/f'{readme_name}{readme_suffix}'
            plugin_additional_readme = Path(doc_path)/f'{readme_name}{readme_additional}{readme_suffix}'
        except IndexError:
            print(f'   skipping {plugin} due to incomplete documentation')
            continue
        
        # pull the help information for the plugin
        readme_text = get_help.get_help(plugin, False, plugin_path=plugin_path)
        
        # read the additional documentation
        if plugin_additional_readme.exists():
            with open(plugin_additional_readme, 'r') as file:
                additional_text = file.read()
        else:
            additional_text = '' 
        
        default_layout_image = {'filename': 'not found',
                                'path': 'none',
                                'layout_name': 'layout'}
        layout_images = []
        # gather all the images associated with the plugin
        for layout in layouts:
            layout_name = layout['layout']
            image_filename = f'{plugin}.{layout["layout"]}-sample.png'
            image_path = Path(doc_path)/image_filename
                        
            print(f'   processing layout: {layout_name}')
            try:
                image = layout['plugin_obj'].image
            except AttributeError:
                print('  no image available, skipping this layout')
                continue
                
            if (image_path.exists() and overwrite_images) or not image_path.exists():
                print(f'   saving image: {image_path}')
                image.save(image_path)
            else:
                print(f'   image exists, skipping save')
                
            
            
            layout_entry = {'filename': image_filename,
                            'path': image_path,
                            'layout_name': layout_name}
            
            # try to locate the default "layout" to show at the top of the documentation
            if layout_name == 'layout':
                default_layout_image.update(layout_entry)
            
            layout_images.append(layout_entry)

        print('   writing README files')
        with open(plugin_readme, 'w') as file:
            plugin_name = ".".join(doc_path.parts)
            file.write(f'# {plugin}\n')
            file.write(f'![sample image for plugin {plugin_name}](./{default_layout_image["filename"]}) \n\n')
            file.write(f'```\n{readme_text}\n```\n\n')
            file.write(f'## Provided Layouts:\n\n')
            for l in layout_images:
                file.write(f'layout: **{l["layout_name"]}**\n\n')  
                file.write(f'![sample image for plugin {l["layout_name"]}](./{l["filename"]}) \n\n\n')

            file.write(additional_text)
        
        plugin_docs[plugin] = {'readme': plugin_readme, 'image': default_layout_image["path"]}
            
    
    return plugin_docs

In [10]:
def update_plugin_docs(plugin_docs, doc_path):
    '''update Plugin.md documentation with snips from all plugin READMEs
    
    Args:
        plugin_docs(dict): dictionary provided by create_readme'''
    doc_path = Path(doc_path)
    plugin_readme_source = Path(doc_path/'source/Plugins.md')
    plugin_readme_post_source = Path(doc_path/'source/Plugins_post.md')
    plugin_readme_final = Path(doc_path/plugin_readme_source.name)
    print('updating Plugins.md...')
    print(f'using: {plugin_readme_source}')
    print(f'using postscript file: {plugin_readme_post_source}')
    with open(plugin_readme_source, 'r') as file:
        source = file.read()
        
    with open(plugin_readme_post_source, 'r') as file:
        post = file.read()
    
    with open(plugin_readme_final, 'w') as file:
        file.write(source)
        for plugin, values in plugin_docs.items():
            file.write(f'### [{plugin}]({Path("..")/values["readme"]})\n')
            file.write(f'![{plugin} sample Image]({Path("..")/values["image"]})\n\n')
            
        file.write(post)        

In [12]:
def main():
    parser = argparse.ArgumentParser(description='create_docs')
 
    parser.add_argument('-o', '--overwrite_images', default=False, action='store_true',
                       help='overwrite existing images for plugins when updating README files')
    
    parser.add_argument('-p', '--plugin_list', default=None, nargs='*', 
                       help='list of specific plugins to process')
    
    parser.add_argument('-r', '--project_root', default='./paperpi/', nargs=1,
                       help='path to project root')
    
    parser.add_argument('-d', '--documentation_path', default='./documentation',
                       help='path to documentation directory')
    
    args = parser.parse_args()
#     args = parser.parse_known_args()
    plugin_dict = setup_plugins(args.project_root, args.plugin_list)
    plugin_docs = create_readme(plugin_dict, 
                                project_root=args.project_root,
                                overwrite_images=args.overwrite_images)
    update_plugin_docs(plugin_docs, doc_path=args.documentation_path)
    

In [13]:
if __name__ == "__main__":
    if '-f' in sys.argv:
        logging.debug('looks like this is running in a Jupyter notebook')
        idx = sys.argv.index('-f')
        del sys.argv[idx:idx+2]    
    logger.setLevel('DEBUG')
    logging.root.setLevel('DEBUG')
    print('updating documents...')
    main()

DEBUG:root:importing pkg: paperpi.plugins
DEBUG:root:[[----checking default values for layout----]
DEBUG:root:section: [-----------bin_img------------]
DEBUG:root:adding "dimensions: None"
DEBUG:root:adding "mode: 1"
DEBUG:root:adding "fill: 0"
DEBUG:root:adding "bkground: 255"
DEBUG:root:section: [-------------time-------------]
DEBUG:root:adding "maxchar: None"
DEBUG:root:adding "dimensions: None"
DEBUG:root:adding "mode: 1"
DEBUG:root:adding "fill: 0"
DEBUG:root:adding "bkground: 255"
DEBUG:root:[[....calculating layouts....]]
INFO:root:section: [...........bin_img............]
DEBUG:root:resolution: (640, 400)
DEBUG:root:width: 1, height: 0.8888888888888888
DEBUG:root:absolute coordinates provided
DEBUG:root:block coordinates: (0, 0)
INFO:root:section: [.............time.............]
DEBUG:root:resolution: (640, 400)
DEBUG:root:width: 1, height: 0.1111111111111111
DEBUG:root:scaling font size
DEBUG:root:y target size reached
DEBUG:root:calculated font size: 26
DEBUG:root:calculati

updating documents...
found up plugin dec_binary_clock
adding layout: dec_binary_clock


DEBUG:root:maximum characters per line: 53
DEBUG:root:formatting string: NONE
DEBUG:root:text size: (50, 31)
DEBUG:root:pasting using random coordinates
DEBUG:root:paste coordinates: (45, 7)
DEBUG:root:TIME = None
DEBUG:root:resizing image to fit area: [630, 346]
DEBUG:root:new image size: (381, 346)
DEBUG:root:padding: 5
DEBUG:root:formatting string: 21:49
DEBUG:root:text size: (54, 31)
DEBUG:root:pasting using random coordinates
DEBUG:root:paste coordinates: (363, 5)
DEBUG:root:[[----checking default values for layout----]
DEBUG:root:section: [-----------bin_img------------]
DEBUG:root:section: [-------------time-------------]
DEBUG:root:[[....calculating layouts....]]
INFO:root:section: [...........bin_img............]
DEBUG:root:resolution: (640, 400)
DEBUG:root:width: 1, height: 0.8888888888888888
DEBUG:root:absolute coordinates provided
DEBUG:root:block coordinates: (0, 0)
INFO:root:section: [.............time.............]
DEBUG:root:resolution: (640, 400)
DEBUG:root:width: 1, h

adding layout: layout


DEBUG:root:maximum characters per line: 53
DEBUG:root:formatting string: NONE
DEBUG:root:text size: (50, 31)
DEBUG:root:pasting using random coordinates
DEBUG:root:paste coordinates: (196, 6)
DEBUG:root:TIME = None
DEBUG:root:resizing image to fit area: [630, 346]
DEBUG:root:new image size: (381, 346)
DEBUG:root:padding: 5
DEBUG:root:formatting string: 21:49
DEBUG:root:text size: (54, 31)
DEBUG:root:pasting using random coordinates
DEBUG:root:paste coordinates: (579, 6)
DEBUG:root:importing pkg: paperpi.plugins
DEBUG:root:[[----checking default values for layout----]
DEBUG:root:section: [------forecast_location-------]
DEBUG:root:adding "font_size: None"
DEBUG:root:adding "maxchar: None"
DEBUG:root:adding "dimensions: None"
DEBUG:root:adding "fill: 0"
DEBUG:root:adding "bkground: 255"
DEBUG:root:section: [------time_updated_local------]
DEBUG:root:adding "font_size: None"
DEBUG:root:adding "maxchar: None"
DEBUG:root:adding "inverse: False"
DEBUG:root:adding "dimensions: None"
DEBUG:roo

...

adding layout: word_clock


DEBUG:root:y target size reached
DEBUG:root:calculated font size: 29
DEBUG:root:calculating block position from relative positions
DEBUG:root:block coordinates: (0, 343)
INFO:root:[[____setting blocks____]]
INFO:root:section: [___________wordtime___________]
DEBUG:root:setting block type: TextBlock
DEBUG:root:block area: (640, 343)
DEBUG:root:padded area: [620, 323]
DEBUG:root:fill: 255, bkground: 0
DEBUG:root:creating Block
DEBUG:root:setting old_font = None
DEBUG:root:calculating maximum characters for font ('Anton', 'Regular')
DEBUG:root:average character width: 37.52022244691608
DEBUG:root:maximum characters per line: 17
DEBUG:root:formatting string: NONE
DEBUG:root:text size: (156, 99)
DEBUG:root:pasting using random coordinates
DEBUG:root:paste coordinates: (301, 194)
INFO:root:section: [_____________time_____________]
DEBUG:root:setting block type: TextBlock
DEBUG:root:block area: (640, 57)
DEBUG:root:padded area: [620, 37]
DEBUG:root:fill: 255, bkground: 0
DEBUG:root:creating B

adding layout: word_clock_lg


DEBUG:root:average character width: 41.114256825075834
DEBUG:root:maximum characters per line: 15
DEBUG:root:formatting string: NONE
DEBUG:root:text size: (171, 109)
DEBUG:root:pasting using random coordinates
DEBUG:root:paste coordinates: (126, 212)
INFO:root:section: [_____________time_____________]
DEBUG:root:setting block type: TextBlock
DEBUG:root:block area: (640, 40)
DEBUG:root:padded area: [620, 20]
DEBUG:root:fill: 255, bkground: 0
DEBUG:root:creating Block
DEBUG:root:setting old_font = None
DEBUG:root:calculating maximum characters for font ('Anton', 'Regular')
DEBUG:root:average character width: 6.741658240647118
DEBUG:root:maximum characters per line: 92
DEBUG:root:formatting string: NONE
DEBUG:root:text size: (27, 18)
DEBUG:root:pasting using random coordinates
DEBUG:root:paste coordinates: (119, 10)
INFO:root:update_function for None
DEBUG:root:using 21:50
DEBUG:root:formatting string: It is almost Ten 'Til Ten
DEBUG:root:text size: (402, 219)
DEBUG:root:pasting using ran

Processing plugins: dec_binary_clock
   processing layout: dec_binary_clock
   image exists, skipping save
   processing layout: layout
   image exists, skipping save
   writing README files
Processing plugins: met_no
   processing layout: layout
   image exists, skipping save
   processing layout: three_column_icon_wind_temp_precip
   image exists, skipping save
   processing layout: three_row_icon_wind_temp
   image exists, skipping save
   processing layout: two_column_icon_wind_temp_precip
   image exists, skipping save
   writing README files
Processing plugins: moon_phase
   processing layout: layout
   image exists, skipping save
   processing layout: moon_data
   image exists, skipping save
   processing layout: moon_only
   image exists, skipping save
   writing README files
Processing plugins: announce_update
   skipping announce_update due to incomplete documentation
Processing plugins: default
   processing layout: default
   image exists, skipping save
   processing layout

In [ ]:
!jupyter-nbconvert --to python --template python_clean create_docs.ipynb